# Dependency

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import patsy
from sklearn.base import clone
from sklearn.linear_model import LassoCV, LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder

import statsmodels.formula.api as smf
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_predict
from doubleml.datasets import fetch_401K


In [32]:
from doubleml import DoubleMLData
from doubleml import DoubleMLPLR, DoubleMLIRM

# Data

In [3]:
raw = pd.read_csv('~/Desktop/projects/datasets/cops_df.csv')
raw

,pid,hh_id,Z,D,R_t0,R_t1,R_t2,itrust_care_t0,itrust_care_t1,itrust_care_t2,...,perform_index_t1,perform_index_t2,primary_dvs_index_t0,primary_dvs_index_t1,primary_dvs_index_t2,csi_index_t0,csi_index_t1,csi_index_t2,trustgov_local_index_t0,trustgov_local_index_t1
0,43faea1c,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5f73f3c1,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,92e1aa04,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,e82b9dcf,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,336383dc,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49752,b1d4a705,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49753,5511db00,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49754,07e5ee67,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49755,22458bda,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
covariates = ["dem_age", "dem_female", "dem_race4", "dem_pid7", "dem_inc_high", 
              "contact_arrest_t0", "contact_unfair_t0", "contact_f2f_t0", "spanish"]

In [5]:
data = raw[covariates + ['Z', 'D' ,'primary_dvs_index_t1', 'primary_dvs_index_t0']]

# Prep data

Need to one hot encode the categorical variable dem_rave4 for the tree models.

In [102]:
enc = OneHotEncoder(handle_unknown='ignore')
ecoded_dem_race4 = enc.fit_transform(raw[['dem_race4']]).toarray()

ecoded_dem_race4_df = pd.DataFrame(ecoded_dem_race4, columns=['race_black', 'race_hispanic', 'race_other', 'race_white', 'race_none'])

In [103]:
data = pd.concat(
    [
        raw[["dem_age", "dem_female", "dem_pid7", "dem_inc_high", "dem_race4",
              "contact_arrest_t0", "contact_unfair_t0", "contact_f2f_t0", "spanish"] + ['Z', 'D' ,'primary_dvs_index_t1', 'primary_dvs_index_t0']],
        ecoded_dem_race4_df
    ],
    axis=1
)

# OLS

In [10]:
# For intent to treat (ITT) 

# Define the variables
y = "primary_dvs_index_t1"
X = ["primary_dvs_index_t0"] + covariates

# Define formula
X_str = " + ".join(X)
formula = f"{y} ~ Z + {X_str}"

# Model
model = smf.ols(formula=formula, data=data).fit() # There might be an option to use robust standard errors
model = model.get_robustcov_results(cov_type='HC1', use_t=None)
print(model.summary())

                             OLS Regression Results                             
Dep. Variable:     primary_dvs_index_t1   R-squared:                       0.761
Model:                              OLS   Adj. R-squared:                  0.759
Method:                   Least Squares   F-statistic:                     364.6
Date:                  Sat, 29 Apr 2023   Prob (F-statistic):               0.00
Time:                          08:26:41   Log-Likelihood:                -5199.9
No. Observations:                  1484   AIC:                         1.043e+04
Df Residuals:                      1470   BIC:                         1.050e+04
Df Model:                            13                                         
Covariance Type:                    HC1                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept 

# Double ML - ATE

In [11]:
params = {
    'max_depth': 3,
}
y = "primary_dvs_index_t1"
d = "Z"
X = ["dem_age", "dem_female", "dem_pid7", "dem_inc_high", "race_black",
       "race_hispanic", "race_other", "race_white", "race_none",
              "contact_arrest_t0", "contact_unfair_t0", "contact_f2f_t0", "spanish", "primary_dvs_index_t0"]


# Machine learning models
debias_m = LGBMRegressor(**params)
denoise_m = LGBMRegressor(**params)

# Get residuals for y and T
data_pred = data.assign(d_res =  data[d] - cross_val_predict(debias_m, data[X], data[d], cv=5),
                          y_res =  data[y] - cross_val_predict(denoise_m, data[X], data[y], cv=5))

# Predict the average treatment effect 
dml_ml = smf.ols(formula='y_res ~ d_res', data=data_pred).fit()
dml_ml.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,16.7508,0.306,54.822,0.000,16.151,17.350
d_res,2.6657,0.603,4.417,0.000,1.482,3.849


# Double ML - CATE

In [66]:
dml_ml = smf.ols(formula='y_res ~ d_res * (dem_age + dem_pid7 + dem_inc_high + primary_dvs_index_t0 + C(race_black) + C(race_hispanic) + C(race_other) + C(race_none))', data=data_pred).fit()
dml_ml.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,-3.7679,1.329,-2.835,0.005,-6.375,-1.161
C(race_black)[T.1.0],7.5265,0.750,10.030,0.000,6.055,8.998
C(race_hispanic)[T.1.0],9.1549,0.924,9.909,0.000,7.343,10.967
C(race_other)[T.1.0],7.8455,0.917,8.552,0.000,6.046,9.645
C(race_none)[T.1.0],2.599e-14,1.11e-14,2.344,0.019,4.24e-15,4.77e-14
d_res,5.5887,2.614,2.138,0.033,0.460,10.717
d_res:C(race_black)[T.1.0],2.6171,1.479,1.770,0.077,-0.284,5.518
d_res:C(race_hispanic)[T.1.0],3.9010,1.787,2.183,0.029,0.396,7.406
d_res:C(race_other)[T.1.0],0.6269,1.836,0.341,0.733,-2.974,4.228
d_res:C(race_none)[T.1.0],4.199e-15,6.44e-16,6.524,0.000,2.94e-15,5.46e-15


In [67]:
dml_ml.predict(X)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/S/Desktop/projects/causal_inference/.venv/lib/python3.9/site-packages/patsy/compat.py", line 36, in call_and_wrap_exc
    return f(*args, **kwargs)
  File "/Users/S/Desktop/projects/causal_inference/.venv/lib/python3.9/site-packages/patsy/eval.py", line 169, in eval
    return eval(code, {}, VarLookupDict([inner_namespace]
  File "<string>", line 1, in <module>
  File "/Users/S/Desktop/projects/causal_inference/.venv/lib/python3.9/site-packages/patsy/eval.py", line 52, in __getitem__
    return d[key]
  File "/Users/S/Desktop/projects/causal_inference/.venv/lib/python3.9/site-packages/patsy/eval.py", line 52, in __getitem__
    return d[key]
TypeError: list indices must be integers or slices, not str

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/S/Desktop/projects/causal_inference/.venv/lib/python3.9/site-packages/statsmodels/base/model.py", line 1137, in predict

# Double ML - Cate with DoubleML Library

In [24]:
# Define the variables
y = "primary_dvs_index_t1"
d = "Z"
X = ["dem_age", "dem_female", "dem_pid7", "dem_inc_high", "race_black",
       "race_hispanic", "race_other", "race_white", "race_none",
              "contact_arrest_t0", "contact_unfair_t0", "contact_f2f_t0", "spanish", "primary_dvs_index_t0"]

# The doublem library doesn't allow nan y or d values
filtered_data = data[~data[y].isnull()].reset_index(drop=True)

In [26]:
# Initialize data object
dml_data = DoubleMLData(filtered_data, y_col=y, d_cols=d, x_cols=X, force_all_x_finite=False)

In [31]:
# ATE
ml_g = RandomForestRegressor(n_estimators = 500, max_features = 'sqrt', max_depth= 5)
ml_m = RandomForestRegressor(n_estimators = 500, max_features = 'sqrt', max_depth= 5)

dml_plr = DoubleMLPLR(dml_data, ml_g, ml_m).fit()

print(dml_plr)

================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: primary_dvs_index_t1
Treatment variable(s): ['Z']
Covariates: ['dem_age', 'dem_female', 'dem_pid7', 'dem_inc_high', 'race_black', 'race_hispanic', 'race_other', 'race_white', 'race_none', 'contact_arrest_t0', 'contact_unfair_t0', 'contact_f2f_t0', 'spanish', 'primary_dvs_index_t0']
Instrument variable(s): None
No. Observations: 1484

------------------ Score & algorithm ------------------
Score function: partialling out
DML algorithm: dml2

------------------ Machine learner   ------------------
Learner ml_l: RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=500)
Learner ml_m: RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=500)
Out-of-sample Performance:
Learner ml_l RMSE: [[10.60948239]]
Learner ml_m RMSE: [[0.50315238]]

------------------ Resampling        ------------------
No. folds: 5
No. repeated sample split

# GATE

In [109]:
# Define the variables
y = "primary_dvs_index_t1"
d = "Z"
X = ["dem_age", "dem_female", "dem_pid7", "dem_inc_high", "race_black",
       "race_hispanic", "race_other", "race_white", "race_none",
              "contact_arrest_t0", "contact_unfair_t0", "contact_f2f_t0", "spanish", "primary_dvs_index_t0"]

# The doublem library doesn't allow nan y or d values
filtered_data = data[~data[y].isnull()].reset_index(drop=True)

# Initialize data object
dml_data = DoubleMLData(filtered_data, y_col=y, d_cols=d, x_cols=X, force_all_x_finite=False)

In [110]:
ml_g = RandomForestRegressor(n_estimators = 500, max_features = 'sqrt', max_depth= 5)
ml_m = RandomForestClassifier(n_estimators=100, max_features=20, max_depth=5, min_samples_leaf=2)

dml_irm = DoubleMLIRM(dml_data, ml_g, ml_m).fit()

print(dml_irm)

================== DoubleMLIRM Object ==================

------------------ Data summary      ------------------
Outcome variable: primary_dvs_index_t1
Treatment variable(s): ['Z']
Covariates: ['dem_age', 'dem_female', 'dem_pid7', 'dem_inc_high', 'race_black', 'race_hispanic', 'race_other', 'race_white', 'race_none', 'contact_arrest_t0', 'contact_unfair_t0', 'contact_f2f_t0', 'spanish', 'primary_dvs_index_t0']
Instrument variable(s): None
No. Observations: 1484

------------------ Score & algorithm ------------------
Score function: ATE
DML algorithm: dml2

------------------ Machine learner   ------------------
Learner ml_g: RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=500)
Learner ml_m: RandomForestClassifier(max_depth=5, max_features=20, min_samples_leaf=2)
Out-of-sample Performance:
Learner ml_g0 RMSE: [[10.39870375]]
Learner ml_g1 RMSE: [[10.91561136]]
Learner ml_m RMSE: [[0.5008365]]

------------------ Resampling        ------------------
No. folds: 5
No

In [171]:
groups = pd.DataFrame(np.column_stack([filtered_data['dem_female'] == 0,
                                       filtered_data['dem_female'] == 1
                                       ]),
             columns=['Group 1', 'Group 2'])
groups.head()

,Group 1,Group 2
0,True,False
1,False,True
2,True,False
3,True,False
4,True,False


In [172]:
dml_irm.gate(groups=groups)
print(gate.confint(level=0.95))

            2.5 %    effect    97.5 %
Group 1  2.793605  3.125133  3.456661
Group 2  0.935483  1.233083  1.530683
Group 3 -0.170899  0.024477  0.219853
